3. Predict all news data which is in folder normalized/ --- again just need date, news source, and then all the columns (we want column names of prediction as labels than code)

In [1]:
import pandas as pd
import numpy as np
import time
from glob import glob

## Read news dataset

In [2]:
import nltk
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer
import re
import string

stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

#def tokenize(text):
#    tokens = nltk.word_tokenize(text)
#    stems = stem_tokens(tokens, stemmer)
#    return stems

def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [3]:
from nltk.corpus import stopwords

def clean_text(text):
    text = text.decode('ascii', 'ignore')
    text = re.sub('\d+', '', text)
    text = text.lower()
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    stems = stem_tokens(tokens, stemmer)
    return ' '.join(stems)


In [9]:
import os
ban_files = ['all-news.csv']
files = {}
count = 0
for f in glob('../normalized/*.csv'):
    if os.path.basename(f) in ban_files:
        continue
    df = pd.read_csv(f, nrows=10)
    if 'text' not in df.columns:
        text = 'Content'
        date = 'Date'
        files[f] = {'usecols': ['Date', 'Content', 'channel_reg'], 'parse_dates': {'date': [0]}}
    else:
        files[f] = {'usecols': ['year', 'month', 'day', 'text', 'channel_reg'], 'parse_dates': {'date': [1, 2, 3]}}
    count += 1
print count


9


In [11]:
total = 0    
for i, f in enumerate(files.keys()):
    print f
    NROWS = 20000
    reader = pd.read_csv(f, iterator=True, chunksize=NROWS, **files[f])
    for df in reader:
        if 'Content' in df.columns:
            df = df[['date', 'channel_reg', 'Content']]
        df.columns = ['date', 'news_source', 'text']
        if 'msnbc-reg.csv' in f:
            df['text'] = df.text.astype(str).apply(lambda c: clean_text(c))
        else:
            df.text = df.text.str.replace(r'\d+', '')
        if total == 0:
            with open("../normalized/all-news.csv", "w") as fp:
                df.to_csv(fp, index=False)
        else:
            with open("../normalized/all-news.csv","a") as fp:
                df.to_csv(fp, header=False, index=False)
        #print df
        total += 1
        print total
        #break
    #break

../normalized/newsbank-cnn-fn-cleaned-reg.csv
1
../normalized/lacc-cleaned-reg.csv
2
3
4
5
6
7
8
9
../normalized/cnn-cleaned-reg.csv
10
11
12
13
14
15
16
17
18
19
../normalized/newsbank-stf-fox-cleaned-reg.csv
20
21
../normalized/msnbc-reg.csv
22
../normalized/newsbank-cnn-inter-cleaned-reg.csv
23
../normalized/newsbank-fox-cleaned-reg.csv
24
../normalized/newsbank-msnbc-cleaned-reg.csv
25
../normalized/archive-clean-reg.csv
26
27
28
29
30
31
32
33
34
35
36
37
38
39
